<a href="https://colab.research.google.com/github/michele1993/Protein_design/blob/main/PropGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import sys
sys.path.append('content')
import os
import pandas as pd
import numpy as np


## Data clearning
Need to clean the data and split them in training and validation dataset for fine-tuning PropGPT2

In [10]:
# Load data
#root_dir = os.path.dirname(os.path.abspath(__file__))
file_path = os.path.join('dataset','sequences.csv')

dataset = pd.read_csv(file_path)

In [ ]:
## ------ 1st Remove any pair if contains NaN -----
# identify any row with NaN
data_nan = pd.isna(dataset).sum(axis=1).astype('bool') # sum to identify if there is at least one NaN entry in a row.
# remove entries with NaN entirely
data_cleaned = dataset.loc[~data_nan, :]

assert ~pd.isna(data_cleaned).any().any(), "There are NaN entries in the data, need cleaning"
## ----------------------------

In [ ]:
## ------ 2nd Remove any duplicate entry ----

# try adding a duplicate to test if works:
#data_cleaned = pd.concat([data_cleaned, pd.DataFrame(data_cleaned.iloc[-1,:], columns=data_cleaned.columns)], ignore_index=True)
#data_cleaned.loc[len(data_cleaned.index)] = data_cleaned.iloc[0,:]
#print(data_cleaned.shape)

# find all duplicates:
#duplicates = data_cleaned[data_cleaned.duplicated(subset="mutated_sequence", keep=False)]

# Remove duplicates by only keeping 'first' occurance for each
data_cleaned = data_cleaned.drop_duplicates(subset="mutated_sequence", keep="first")
#print(data_cleaned.shape)
## -------------------------------------------

In [ ]:
## ------- 3rd Prepara data for fine-tuning PropGPT2 on this dataset -----
# for the moment ingore the activations just with all sequences
data_cleaned_seq = data_cleaned.iloc[:,0]

# 1st need to add "<|endoftext|>" token at the beginning of each seq
special_token = "<|endoftext|>"
data_cleaned_seq = special_token + data_cleaned_seq

# 2nd need to slip the data in training and validation

# Select % of validation seqs (i.e., 90/10)
n_seq = data_cleaned_seq.shape[0]
n_validation = n_seq // 10

# Select n. random indexes for validation
val_indx = np.random.randint(0, n_seq, n_validation)
# Select validation seqs
val_seq = data_cleaned_seq.iloc[val_indx]
# Select training seqs by eliminating validation seqs
training_seq = data_cleaned_seq.drop(index=val_indx)

# 3rd: concatane all strings together and save in a txt file
training_concatenated = ''.join(training_seq)
val_concatenated = ''.join(val_seq)

# to add newline character between each original row's string use
#concatenated = '\n'.join(training_seq)

# Save concatenated strings
with open('training.txt','w') as file:
    file.write(training_concatenated)

with open('validation.txt','w') as file:
    file.write(val_concatenated)

## Now, we can try to fine-tune PropGPT2 based on our data

First we need to install some dependecies

In [19]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 235896, done.
remote: Counting objects: 100% (21606/21606), done.
remote: Compressing objects: 100% (1527/1527), done.
remote: Total 235896 (delta 21130), reused 20112 (delta 20054), pack-reused 214290 (from 1)
Receiving objects: 100% (235896/235896), 241.61 MiB | 20.15 MiB/s, done.
Resolving deltas: 100% (172884/172884), done.
Updating files: 100% (4703/4703), done.


In [23]:
!pip install -e transformers/.

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.9 MB/s eta 0:00:00
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.46.0.dev0-0.editable-py3-none-any.whl size=17265 sha256=494b9e2eb8c745bcc4accf621296c56e2e062106bea20934626ac88befc11f97
  Stored in directory: /tmp/pip-ephem-wheel-cache-q2v3itne/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstallin

In [16]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
! python run_clm.py --model_name_or_path nferruz/ProtGPT2 --train_file training.txt --validation_file validation.txt --tokenizer_name nferruz/ProtGPT2 --do_train --do_eval --output_dir output --learning_rate 1e-06

2024-10-18 17:36:21.374819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 17:36:21.409147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 17:36:21.419250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-18 17:36:21.442999: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-18 17:36:23.168103: W tensorflow/compiler/tf2